# Robot de Limpieza Reactivo
Este modelo estudia las estadísticas de un robot de limpieza reactivo, donde se recopila la siguiente información:
* Tiempo necesario hasta que todas las celdas estén limpias (o se haya llegado al tiempo máximo).
* Porcentaje de celdas limpias después del termino de la simulación.
* Número de movimientos realizados por todos los agentes.

## Datos proporcionados
* Habitación de MxN espacios
* Número de agentes
* Porcentaje de celdas inicialmente sucias
* Tiempo máximo de ejecución

## Reglas
Las reglas del modelo son:
* Inicializa las celdas sucias (ubicaciones aleatorias).
* Todos los agentes empiezan en la celda [1,1].
* En cada paso de tiempo:
    * Si la celda está sucia, entonces aspira.
    * Si la celda está limpia, el agente elije una dirección aleatoria para moverse (unas de las 8 celdas vecinas) y elije la acción de movimiento (si no puede moverse allí, permanecerá en la misma celda)
    * Se ejecuta el tiempo máximo estable

Para un espacio de 100x100, considera los siguientes escenarios:
* Escenario 1: 1 agente, 90% de celdas sucias
* Escenario 2. 2 agentes, 90% de celdas sucias

Deberás resolver las siguientes preguntas:
* ¿Cuántos pasos de simulación toma limpiar todo el espacio?
* ¿Qué porcentaje de celdas sucias queda con los siguientes pasos de simulación: 100, 1000, 10,000?

A continuación, determina cuál es la cantidad óptima de aspiradoras que debe de tener para realizar la limpieza en el menor tiempo posible. Considera que tenemos un máximo de 10 aspiradoras disponibles.

## Imports

Antes de empezar a crear el modelo del juego de la vida con multiagentes es necesario tener instalado los siguientes paquetes:
- `python`: asegúrense de usar la versión 3+.
- `mesa`: el framework de Python para el modelado de agentes.
- `numpy`: es una biblioteca de Python para el manejo de matrices, arreglos, manipulación matemática, lógica y mucho más.
- `matplotlib`: es una biblioteca para crear visualizaciones estáticas, animadas e interactivas en Python.

Para poder modelar a los integrantes de nuestro sistema usando el framework de `mesa` es necesario importar dos clases: una para el modelo general, y otro para los agentes.

In [509]:
from mesa import Agent, Model
from mesa.time import RandomActivation
from mesa.space import MultiGrid
from mesa.datacollection import DataCollector
from mesa.batchrunner import batch_run

# Importamos los siguientes paquetes para el mejor manejo de valores numéricos.
import numpy as np

# Definimos otros paquetes que vamos a usar para medir el tiempo de ejecución de nuestro algoritmo.
import time as tm
import datetime

In [510]:
class VacuumAgent(Agent):
    def __init__(self, id, model, x, y):
        super().__init__(id, model)
        self.position = (x, y)
        self.movements = 0

    def step(self):
        x_pos = self.position[0]
        y_pos = self.position[1]

        # Clean cell
        if model.dirty_cells[x_pos][y_pos] == 1:
            model.dirty_cells[x_pos][y_pos] = 0

        # Get new position
        possible_cells = self.model.grid.get_neighborhood(self.position, moore=True, include_center = False)
        new_position = self.random.choice(possible_cells)
        cellmate = self.model.grid.get_cell_list_contents([new_position])

        if cellmate:
            return

        self.movements += 1
        self.position = new_position
        self.model.grid.move_agent(self, self.position)

In [511]:
def get_vacuums(model):
    return np.asarray([agent.position for agent in model.schedule.agents])

In [512]:
class VacuumModel(Model):
    def __init__(self, num_agents, width, height, dirty_cells_percent):
        self.schedule = RandomActivation(self)
        self.grid = MultiGrid(width, height, torus = False)
        self.datacollector = DataCollector(model_reporters={"Vacuums": get_vacuums})
        self.dirty_cells = np.zeros((width, height))
        self.dirty_cells_percent = dirty_cells_percent

        # Initialize dirty cells
        init_dc = 0
        while init_dc < int(width * height * (dirty_cells_percent / 100)):
            x = int(np.random.rand() * width)
            y = int(np.random.rand() * height)
            if self.dirty_cells[x][y] == 0:
                self.dirty_cells[x][y] = 1
                init_dc += 1

        # Create agents and place them in [1, 1]
        for i in range(num_agents):
            x = 1
            y = 1
            agent = VacuumAgent(i, self, x, y)
            self.schedule.add(agent)
            self.grid.place_agent(agent, (x, y))

    def step(self):
        self.datacollector.collect(self)
        self.schedule.step()

    def __str__(self):
        total_cells = self.grid.width * self.grid.height
        initial_dirty_cells = int(total_cells * (self.dirty_cells_percent / 100))
        final_dirty_cells = int(self.dirty_cells.sum())
        final_dirty_cells_percent = round(((final_dirty_cells / total_cells) * 100), 2)
        clean_cells = int(total_cells - self.dirty_cells.sum())
        clean_cells_percent = round(((clean_cells / total_cells) * 100), 2)
        movements = lambda: sum([agent.movements for agent in self.schedule.agents])

        aux = ""
        aux += f"[TOTAL CELLS]: {total_cells}\n"
        aux += f"[DIRTY CELLS]: {initial_dirty_cells} ({self.dirty_cells_percent}%)\n"
        aux += f"\nAFTER SIMULATION\n"
        aux += f"[DIRTY CELLS]: {final_dirty_cells} ({final_dirty_cells_percent}%)\n"
        aux += f"[CLEAN CELLS]: {clean_cells} ({clean_cells_percent}%)\n"
        aux += f"[MOVEMENTS]: {movements()}\n"

        return str(aux)

In [513]:
WIDTH = 100
HEIGHT = 100
DIRTY_CELLS_PERCENT = 90

In [514]:
def clean(model, iterations, num_vacuums):
  for itr in iterations:
    time = tm.time()

    # Print labels
    print(f"[ITERATIONS]: {itr}")
    print(f"[AGENTS]: {num_vacuums}")

    for _ in range(itr):
        model.step()

    # Print model
    print(model)
    print(f"[TIME]: {round(tm.time() - time, 4)} seconds")
    print(f"----------------------\n")

## Test 1
* NUM_VACUUMS = 1

In [515]:
NUM_VACUUMS = 1
model = VacuumModel(NUM_VACUUMS, WIDTH, HEIGHT, DIRTY_CELLS_PERCENT)

In [516]:
ITERATIONS = [100,1000,10000]
clean(model, ITERATIONS, NUM_VACUUMS)

[ITERATIONS]: 100
[AGENTS]: 1
[TOTAL CELLS]: 10000
[DIRTY CELLS]: 9000 (90%)

AFTER SIMULATION
[DIRTY CELLS]: 8960 (89.6%)
[CLEAN CELLS]: 1040 (10.4%)
[MOVEMENTS]: 100

[TIME]: 0.003 seconds
----------------------

[ITERATIONS]: 1000
[AGENTS]: 1
[TOTAL CELLS]: 10000
[DIRTY CELLS]: 9000 (90%)

AFTER SIMULATION
[DIRTY CELLS]: 8583 (85.83%)
[CLEAN CELLS]: 1417 (14.17%)
[MOVEMENTS]: 1100

[TIME]: 0.0143 seconds
----------------------

[ITERATIONS]: 10000
[AGENTS]: 1
[TOTAL CELLS]: 10000
[DIRTY CELLS]: 9000 (90%)

AFTER SIMULATION
[DIRTY CELLS]: 6645 (66.45%)
[CLEAN CELLS]: 3355 (33.55%)
[MOVEMENTS]: 11100

[TIME]: 0.1621 seconds
----------------------



## Test 2
* NUM_VACUUMS = 2

In [517]:
NUM_VACUUMS = 2
model = VacuumModel(NUM_VACUUMS, WIDTH, HEIGHT, DIRTY_CELLS_PERCENT)

In [518]:
ITERATIONS = [100,1000,10000]
clean(model, ITERATIONS, NUM_VACUUMS)

[ITERATIONS]: 100
[AGENTS]: 2
[TOTAL CELLS]: 10000
[DIRTY CELLS]: 9000 (90%)

AFTER SIMULATION
[DIRTY CELLS]: 8932 (89.32%)
[CLEAN CELLS]: 1068 (10.68%)
[MOVEMENTS]: 196

[TIME]: 0.0061 seconds
----------------------

[ITERATIONS]: 1000
[AGENTS]: 2
[TOTAL CELLS]: 10000
[DIRTY CELLS]: 9000 (90%)

AFTER SIMULATION
[DIRTY CELLS]: 8355 (83.55%)
[CLEAN CELLS]: 1645 (16.45%)
[MOVEMENTS]: 2194

[TIME]: 0.0258 seconds
----------------------

[ITERATIONS]: 10000
[AGENTS]: 2
[TOTAL CELLS]: 10000
[DIRTY CELLS]: 9000 (90%)

AFTER SIMULATION
[DIRTY CELLS]: 5116 (51.16%)
[CLEAN CELLS]: 4884 (48.84%)
[MOVEMENTS]: 22185

[TIME]: 0.2745 seconds
----------------------

